In [5]:
import pandas as pd
import numpy as np
import os
import pickle
import csv

training_examples = []
training_files = [f for f in os.listdir('./training/') if f.endswith('.csv')]
training_files.remove('p108796.csv')  # no dynamic data
training_size = len(training_files)
training_files.sort()
print(training_size)
print(training_files[0])

40335
p000001.csv


In [2]:
static_num = 5
dynamic_num = 34
print(static_num, dynamic_num)

5 34


In [3]:
x = []
static = []
y = []
name = []
mask = []
static_mask = []
max_length = 60

for file in training_files:
    x_one = []
    mask_one = []
    static_one = np.zeros((static_num))
    static_mask_one = np.ones((static_num))
    with open(os.path.join('./training/', file), mode='r', encoding='utf-8') as f:
        reader = csv.reader(f)
        los = None
        for row in reader:
            if row[0] == 'HR':
                continue
            if los == None:
                los = int(row[-2])
            x_row = np.zeros((dynamic_num))
            mask_row = np.zeros((dynamic_num))
            for i in range(dynamic_num):
                if row[i] != '':
                    x_row[i] = float(row[i])
                    mask_row[i] = 1
            x_one.append(x_row)
            mask_one.append(mask_row)
        for i in range(static_num):
            if row[i + dynamic_num] != '':
                static_one[i] = float(row[i + dynamic_num])
            else:
                static_mask_one[i] = 0
        assert int(row[-2]) - los + 1 == len(x_one)
    if len(x_one) >= 8:
        x.append(x_one[:max_length])
        mask.append(mask_one[:max_length])
        static.append(static_one)
        y.append(int(row[-1]))
        name.append(file.split('.')[0])
        static_mask.append(static_mask_one)
print(len(x), len(mask), len(static), len(y), len(name), len(static_mask))

40335 40335 40335 40335 40335 40335


In [4]:
sum(y) / len(y)

0.07269121110697906

In [4]:
static_masked = np.ma.masked_array(static, static_mask == 0)
static_mean = np.mean(static_masked, 0)
static_std = np.std(static_masked, 0)
print(static_mean)
print(static_std)

[61.643364819635195 0.559439692574687 0.3041279285979918
 0.30868972356514196 -51.84894508491751]
[16.48294143717642 0.49645435132206406 0.4600370981177364
 0.4619527877721603 139.76471941192648]


In [5]:
x_flat = []
mask_flat = []
for i in range(len(x)):
    x_flat += x[i]
    mask_flat += mask[i]
x_masked = np.ma.masked_array(x_flat, np.array(mask_flat) == 0)
x_mean = np.mean(x_masked, 0)
x_std = np.std(x_masked, 0)
print(x_mean)
print(x_std)

[84.40068921212506 97.19541542338456 36.96793197614933 123.49223083462866
 82.26639423301435 63.778894225101425 18.639076486536904 32.64997132675214
 -0.8098003778337529 24.0035282991416 0.5611962938878147 7.377292025639113
 40.97238702301318 92.5940955758454 260.27122937777403 23.402473641124775
 101.4817485830302 7.562737074699891 105.87319338270426 1.503799489286105
 1.7383651804670983 136.88015798308658 2.6781216776458234
 2.044774645455106 3.5451515306470016 4.141621221621143 1.9137104484407141
 8.327994920373143 30.847752073264314 10.457268079823152
 41.090147814909926 11.428119707690627 285.35315203955497
 196.3631885870183]
[17.261695519395825 2.9359194472523096 0.7656289517210707
 23.05204325470559 16.31529125333787 13.952064430846697 5.041665018524193
 7.971572686781445 4.273004765636656 4.348948699776841 11.625811344559338
 0.07450378537197386 9.227759710771865 10.979302689448696 857.928375137658
 19.64150408680875 121.40877635839475 2.4203115046013752 5.849430016059299
 1.8

In [6]:
x_normalize = []
static_normalize = []
mask_normalize = []
for i in range(len(x)):
    static_normalize.append(np.where(static_mask[i] > 0, (static[i] - static_mean) / static_std, 0).tolist())
    x_one = []
    mask_one = []
    for j in range(len(x[i])):
        x_one.append(np.where(mask[i][j] > 0, (x[i][j] - x_mean) / x_std, 0).tolist())
        mask_one.append(np.where(mask[i][j] > 0, 1, 0).tolist())
    x_normalize.append(x_one)
    mask_normalize.append(mask_one)

In [ ]:
index_array = list(range(len(x)))
# np.random.shuffle(index_array)
print(index_array)

In [8]:
x_random = []
y_random = []
static_random = []
mask_random = []
name_random = []
for i in range(len(index_array)):
    x_random.append(x_normalize[index_array[i]])
    y_random.append(y[index_array[i]])
    static_random.append(static_normalize[index_array[i]])
    mask_random.append(mask_normalize[index_array[i]])
    name_random.append(name[index_array[i]])

In [9]:
pickle.dump((x_random, y_random, static_random, mask_random, name_random), open('data_normalized.pkl', 'wb'))
stat_dict = {'x_mean': x_mean, 'x_std': x_std, 'static_mean': static_mean, 'static_std': static_std}
pickle.dump(stat_dict, open('stat.pkl', 'wb'))

In [6]:
cnt = 0
cnt1 = 0
for i in range(len(mask)):
    for j in range(len(mask[i])):
        cnt += sum(mask[i][j])
        cnt1 += len(mask[i][j])
print('Observed Rate:', cnt / cnt1)

Observed Rate: 0.19806265372723786
